# Discussion 07 Notebook

This notebook is an accompaniment to the associated discussion worksheet handout.

## Database Setup

In [1]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS disc07'
!psql -h localhost -c 'CREATE DATABASE disc07'

%reload_ext sql
%sql postgresql://127.0.0.1:5432/disc07

NOTICE:  database "disc07" does not exist, skipping
DROP DATABASE
CREATE DATABASE
Deploy FastAPI apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [2]:
!psql postgresql://127.0.0.1:5432/disc07 <disc07.sql

SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
CREATE EXTENSION
COMMENT
SET
SET
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE MATERIALIZED VIEW
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
COPY 6
COPY 176436
COPY 73006
COPY 21
ALTER TABLE
CREATE INDEX
REFRESH MATERIALIZED VIEW


# Section I: Data Granularity

### Initial Exploration

In [3]:
%sql SELECT * FROM nodes ORDER BY tax_id LIMIT 5;

Running query in 'postgresql://127.0.0.1:5432/disc07'

5 rows affected.

tax_id,parent,rank
1,1,no rank
2,131567,superkingdom
6,335928,genus
7,6,species
9,32199,species


In [4]:
%sql SELECT * FROM names ORDER BY tax_id LIMIT 5;

Running query in 'postgresql://127.0.0.1:5432/disc07'

5 rows affected.

tax_id,name_txt
1,all
1,root
2,Bacteria
2,bacteria
2,eubacteria


### Question 1 
Write a SQL query to find the node representing the Animalia kingdom. Your query should return
the kingdom name and its corresponding node information.

In [7]:
%%sql
-- your code here


Running query in 'postgresql://127.0.0.1:5432/disc07'

RuntimeError: (psycopg2.ProgrammingError) can't execute an empty query
[SQL: -- your code here]
(Background on this error at: https://sqlalche.me/e/20/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


### Questoin 2
Let us drill down into the _Animalia_ kingdom. First, find all children nodes of the Animalia kingdom
(ID 33208). Your query should return the `tax_id` and `rank` of each child node. Hint: use self-join.

In [ ]:
%%sql
-- your code here


### Question 3
Next, find the names of these children nodes, along with the names of their parents. Your query should
return the `tax_id`, `rank`, and `name_txt` of each child and parent node.

In [ ]:
%%sql
-- your code here


### Question 4
In the biological taxonomy data, it is common for the same phylum to have multiple synonym names,
all sharing the same tax id. Write a SQL query to return the parent node (Animalia) and its direct
child nodes (phyla). For each phylum (child node), aggregate all its synonym names into a single JSON
array.

In [19]:
%%sql
-- your code here


Running query in 'postgresql://127.0.0.1:5432/disc07'

RuntimeError: (psycopg2.ProgrammingError) can't execute an empty query
[SQL: -- your code here]
(Background on this error at: https://sqlalche.me/e/20/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


### **Challenge:** Question 5  
How can we drill down one more layer? What if we want to get the names of all the classes under the Animalia kingdom?

In [ ]:
%%sql
-- your code here


# Section II: Recursive Queries

The Fibonacci sequence is a famous series of numbers where each number is the sum of the two preceding ones. The sequence appears as follows:

$$0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, ...$$

Mathematically, the n-th Fibonacci number is defined by:
$$F_n = F_{n-1} + F_{n-2}$$

With base cases:

$$F_1 = 0$$
$$F_2 = 1$$

In [10]:
import pandas as pd

# Data
fib_data = {
    "n": [1, 2, 3, 4, 5, 6, 7],
    "Fibonacci Number": [0, 1, 1, 2, 3, 5, 8]
}

fib_df = pd.DataFrame(fib_data)

fib_df


,n,Fibonacci Number
0,1,0
1,2,1
2,3,1
3,4,2
4,5,3
5,6,5
6,7,8


### Question 6
Write a recursive SQL query to compute the 10th Fibonacci number using the `WITH RECURSIVE` statement.

In [14]:
%%sql
-- Your code here


Running query in 'postgresql://127.0.0.1:5432/disc07'

RuntimeError: (psycopg2.ProgrammingError) can't execute an empty query
[SQL: -- Your code here]
(Background on this error at: https://sqlalche.me/e/20/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


# Section III: Window Functions

## Load in the IMDB Performance database

This is a variation of the IMDB database with keys defined. Note that this is a pretty big database! So if you run the below lines, please also remember to delete the `imdb_perf_lecture` afterwards to save space on your limited postgreSQL server.

In [20]:
!ln -sf ../../proj/proj1/data .
!unzip -u data/imdbdb.zip -d data/

Archive:  data/imdbdb.zip


In [21]:
!psql postgresql://jovyan@127.0.0.1:5432/imdb -c 'SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity WHERE datname = current_database()  AND pid <> pg_backend_pid();'
!psql postgresql://jovyan@127.0.0.1:5432/postgres -c 'DROP DATABASE IF EXISTS imdb'
!psql postgresql://jovyan@127.0.0.1:5432/postgres -c 'CREATE DATABASE imdb'
!psql postgresql://jovyan@127.0.0.1:5432/imdb -f data/imdbdb.sql -q

 pg_terminate_backend 
----------------------
(0 rows)

DROP DATABASE
CREATE DATABASE
 set_config 
------------
 
(1 row)



In [22]:
%sql postgresql://jovyan@127.0.0.1:5432/imdb

Connecting and switching to connection postgresql://jovyan@127.0.0.1:5432/imdb

### Question 7
In Project 1, we worked with a movie sample table to explore gross earnings for each movie and stored the results in a view called `movie_gross`. (P.S. We added a `production_year` column here for the subsequent questions)

In [23]:
%%sql
-- How we created movie_gross in Project 1, Question 3a --
DROP VIEW IF EXISTS movie_gross CASCADE;
CREATE VIEW movie_gross AS

WITH cleaned as (
    SELECT
        CAST(REGEXP_REPLACE(SUBSTRING(info, '[0-9,]+'), ',', '', 'g') AS float) AS gross,
        movie_id,
        title,
        production_year
    FROM
        movie_info_sample,
        movie_sample
    WHERE
        movie_id = movie_sample.id AND
        info_type_id = 107 AND
        info LIKE '%(USA)%' and info LIKE '$%'
)

SELECT
    max(gross) AS gross,
    movie_id,
    title,
    production_year
FROM
    cleaned
GROUP BY
    movie_id,
    title,
    production_year
ORDER BY gross DESC;

SELECT * FROM movie_gross
LIMIT 5

Running query in 'postgresql://jovyan@127.0.0.1:5432/imdb'

5 rows affected.

gross,movie_id,title,production_year
760507625.0,1704289,Avatar,2009
658672302.0,2438179,Titanic,1997
623357910.0,2346436,The Avengers,2012
534858444.0,2360583,The Dark Knight,2008
460935665.0,2310522,Star Wars,1977



Write a SQL query to retrieve the top three movies with the highest gross income in the `Action` genre for each year, starting from the year 2000. The result table should include the following columns:
`movie_id`, `title`, `genre`, `production_year`, `rank`, and sorted by `production_year` in ascending order.
Hint: check out the window function `RANK()`.

In [29]:
%%sql
-- Your code here
-- create a CTE here, which gets the genre(s) and gross of each movie --
...

...

Running query in 'postgresql://jovyan@127.0.0.1:5432/imdb'

RuntimeError: If using snippets, you may pass the --with argument explicitly.
For more details please refer: https://jupysql.ploomber.io/en/latest/compose.html#with-argument


Original error message from DB driver:
(psycopg2.errors.SyntaxError) syntax error at or near ".."
LINE 3: ...
        ^

[SQL: -- Your code here
-- create a CTE here, which gets the genre(s) and gross of each movie --
...

...]
(Background on this error at: https://sqlalche.me/e/20/f405)

If you need help solving this issue, send us a message: https://ploomber.io/community


### Question 8
Expanding on discussion 6 Question 3, besdies finding the average gross for movies in each genre but also finding the total gross for each genre. <br/>

Create a view called `movie_avg_total_genre` with the following columns: `movie_id`, `title`, `gross`, `genre`, `avg_gross`,`total_gross`.
- `gross`: The gross earnings of the given movie
- `avg_gross`: The average gross earnings of movies with the same genre as the given movie. If a movie belongs to multiple genres, it should appear in multiple rows, with each row corresponding
to a different genre.
- `total_gross`: The total gross earning of movies with the same genre as the given movie. <br/>

Bonus: Instead of writing out two separate windows, is it possible to simplify?

In [30]:
%%sql
-- Your code here
DROP VIEW IF EXISTS movie_avg_total_genre;
CREATE VIEW movie_avg_total_genre AS 

-- create a CTE here, which gets the genre(s) and gross of each movie --
...

-- for each movie, get its gross and average gross for its genre(s) --
...

SELECT * FROM movie_avg_total_genre
WHERE title = 'Mr. & Mrs. Smith'
ORDER BY genre;

Running query in 'postgresql://jovyan@127.0.0.1:5432/imdb'

RuntimeError: If using snippets, you may pass the --with argument explicitly.
For more details please refer: https://jupysql.ploomber.io/en/latest/compose.html#with-argument


Original error message from DB driver:
(psycopg2.errors.SyntaxError) syntax error at or near ".."
LINE 4: ...
        ^

[SQL: CREATE VIEW movie_avg_total_genre AS 

-- create a CTE here, which gets the genre(s) and gross of each movie --
...

-- for each movie, get its gross and average gross for its genre(s) --
...

SELECT * FROM movie_avg_total_genre
WHERE title = 'Mr. & Mrs. Smith'
ORDER BY genre;]
(Background on this error at: https://sqlalche.me/e/20/f405)

If you need help solving this issue, send us a message: https://ploomber.io/community
